In [33]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.options import Options
import time
from bs4 import BeautifulSoup
import pandas as pd
import urllib
import requests
import re
import random
from database_insta import *

In [37]:
class Insta:
    def __init__(self,username,password):
        self.username = username
        self.password = password
        #self.bot = webdriver.Firefox()
        chrome_options = Options()
        #chrome_options.add_argument("--headless")
        #chrome_options.add_argument("--no-sandbox")

        #chrome_options.add_argument("--disable-gpu")
        self.bot = webdriver.Chrome('chromedriver',options=chrome_options)
        self.bot.get('https://www.instagram.com/akshaykumar/')
    
    def login(self):
        bot = self.bot
        bot.get('https://www.instagram.com/accounts/login/?source=auth_switcher')
        time.sleep(3)
        email = bot.find_element_by_name('username')
        password = bot.find_element_by_name('password')
        email.clear()
        password.clear()
        email.send_keys(self.username)
        password.send_keys(self.password)
        password.send_keys(Keys.RETURN)
        time.sleep(5)
        try:
            notif_dialog = bot.find_element_by_class_name('aOOlW.HoLwm').click()
        except:
            print("dialog not found")
        print("login complete")
    
    def posts(self,hashtag):
        print("def posts start")
        bot = self.bot
        bot.get('https://www.instagram.com/explore/tags/{}/'.format(hashtag))
        time.sleep(2)
        post_links = []
        for i in range(0,10):
            bot.execute_script('window.scrollTo(0,document.body.scrollHeight)')
            time.sleep(2)
            posts = bot.find_elements_by_class_name('v1Nh3.kIKUG._bz0w')
            for post in posts:
                post_links.append(post.find_element_by_css_selector('a').get_attribute('href'))
                print(len(post_links))
        print("post_links len",len(post_links))
        post_links = list(set(post_links))
        print("set post_links",len(post_links))
        f = open('post_links.txt','w')
        for i in post_links:
            f.write(i+'\n')
        f.close()
        #print("post_links len",len(post_links))
        print("def posts end")
    
    def profiles(self):
        bot = self.bot
        print("def profiles start")
        post_links = []
        f = open('post_links.txt','r')
        for i in f:
            post_links.append(i.rstrip())
        f.close()
        profiles =[]
        for i in post_links:
            bot.get(i)
            time.sleep(3)
            try:
                profile = bot.find_element_by_class_name('sqdOP.yWX7d._8A5w5.ZIAjV').get_attribute('href')
                profiles.append(profile)
                print(len(profiles))
            except Exception as e:
                print(e)
        profiles = list(set(profiles))
        print(len(profiles))
        f = open('profiles.txt','w')
        for i in profiles:
            f.write(i+'\n')
        f.close()
        print("def profiles end")
    
    def scraping_profiles(self):
        bot = self.bot
        print('def scraping_profiles start')
        profiles =[]
        f = open('profiles.txt','r')
        for i in f:
            profiles.append(i.rstrip())
        f.close()
        insta_links =[]
        names =[]
        websites =[]
        emails =[]
        for profile in profiles:
            bot.get(profile)
            time.sleep(3)
            try:
                name = bot.find_element_by_class_name('rhpdm').text
                website = bot.find_element_by_class_name('yLUwa').get_attribute('href')
                x = website.find('=')
                y = website.find('&e')
                website = urllib.parse.unquote(website[x+1:y])
                
            except Exception as e:
                print(e)
                #time.sleep(120)
                
                name =''
                website =''
            print(name,website)
            
            insta_links.append(i)
            names.append(name)
            websites.append(website)
            print(len(insta_links),len(names),len(websites))
        
        for i in websites:
            try:
                response = requests.get(i)
                soup = BeautifulSoup(response.text,'html.parser')
                all_text = soup.find('body').getText()
                #print(all_text,type(all_text))
                web_email = list(set(re.findall(r"[a-z0-9\.\-+_]+@[a-z0-9\.\-+_]+\.[a-z0-9\.\-+_]+", all_text, re.I)))
                '''
                if len(web_email)>0:
                    emails.append(web_email[0])
                    print(web_email[0],len(emails))
                else:
                    emails.append('')
                    print("Didnt find",len(emails))
                time.sleep(1)
            except:
                emails.append('')
                print("error",len(emails))
                '''
                if len(web_email) > 0:
                    temp = ''
                    for j in web_email:
                        try:
                            if validate_email(j):
                                emails.append(j)
                                temp = j
                                print(j,len(emails))
                                break
                        except:
                            continue
                    if temp == '':
                        emails.append('')
                        print("Didnt find",len(emails))
                else:
                    emails.append('')
                    print("Didnt find",len(emails))
        
        df = pd.DataFrame({'Business_Name':names,
                          'Instagram_Link':insta_links,
                          'Website':websites,
                          'Email':emails})
        df.to_csv('unfiltered.csv',index=False)
            
        print('def scraping_profiles end')
        return insta_links,names,websites,emails

IndentationError: unexpected unindent (<ipython-input-37-fb4496785a0d>, line 153)

In [28]:
keyword = 'dentist'
s = Insta('_fifty_shades_of_us_','Aryman@235')
#s.login()
#s.posts(keyword)
#s.profiles()
insta_links,names,websites,emails = s.scraping_profiles()

def scraping_profiles start
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
1 1 1
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
2 2 2
Centre Dentaire SMILE WELL http://www.smilewell.ma/
3 3 3
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
4 4 4
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
5 5 5
Indo Dental Center https://www.jd.id/product/indo-dental-center-whitening_611397767/611397768.html
6 6 6
طبيب اسنان وزراعة و تجميل وجه https://karti.me/u/dr-yazan-smadi
7 7 7
Ｒｅｂｅｃｃａ | Ｄｅｎｔａｌ Ｎｕｒｓｅ http://dentalnursepower.blogspot.com/
8 8 8
Hell-Bent® Holsters http://www.hellbentholsters.com/
9 9 9
Message: no such element: Unable 

Sevil Bacak Diş Kliniği http://wa.me/905307906575
91 91 91
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
92 92 92
Love St Andrews http://www.facebook.com/StAndrewsFife/
93 93 93
𝗖𝗲𝗻𝘁𝗿𝗼 𝗱𝗲 𝗣𝗿𝗼́𝘁𝗲𝘀𝗶𝘀 𝗗𝗲𝗻𝘁𝗮𝗹 http://www.centrodeprotesisdentales.com/
94 94 94
Ortodontist Gizem Akgün Karslı https://cutt.ly/uzmdtgizemakgunkarsli
95 95 95
Message: no such element: Unable to locate element: {"method":"css selector","selector":".rhpdm"}
  (Session info: chrome=85.0.4183.83)

 
96 96 96
Dr. Andi-Jean Miro http://aparosenthal.com/
97 97 97
GC Poland http://l.ead.me/bb5peV/
98 98 98
Brand Manager http://ventesmile.com/
99 99 99
Message: no such element: Unable to locate element: {"method":"css selector","selector":".yLUwa"}
  (Session info: chrome=85.0.4183.83)

 
100 100 100
Message: no such element: Unable to locate element: {"method":"css selector","selector":".rhpdm"}
  (Session info: chrome=85.0.4183.8

In [29]:
df = pd.read_csv('unfiltered.csv')
df.shape

(120, 4)

In [30]:
df.dropna(inplace=True)
df.shape

(19, 4)

In [31]:
df.to_csv('filtered.csv',index=False)

In [38]:
db()

Centre Dentaire SMILE WELL https://www.instagram.com/enikokoloszar/
 http://www.smilewell.ma/ cdsmilewell@gmail.com
Ontario Dental Implant Network https://www.instagram.com/enikokoloszar/
 http://livedentalsurgery.com/ info@livedentalsurgery.com
The Professional Plug https://www.instagram.com/enikokoloszar/
 http://www.theprofessionalplug.com/ connect@theprofessionalplug.comCopyright
Gentle Dental & Implant Clinic https://www.instagram.com/enikokoloszar/
 http://www.gentledentalclinic.com/ info@gentledentalclinic.com
Absolute Aesthetics Clinic https://www.instagram.com/enikokoloszar/
 https://dentalhouseliverpool.co.uk/virtual-consultation/ chloe.hale@dentalhouseliverpool.co.uk
Clinica Doc. Belen Lacarte. https://www.instagram.com/enikokoloszar/
 https://www.clinicadentalbelenlacarte.com/ arteestallo@gmail.com
Wilmington Orthodontic Center https://www.instagram.com/enikokoloszar/
 https://www.orthodontistwilmington.com/ info@orthodontistwilmington.com
The Smile Clinic 🦷 https://www.ins